# Improving Machine Learning Experience With GStreamer Techniques
Marco A. Franchi

## *Abstract*

*It has being very common face machine learning algorithms at the multimedia area: traffic count; people count; real-time vigilance cameras; baggage treaker; face/expression recognition; and so on. However, it is perceptive the gap between machine learning and multimedia solutions, where even at simple embedded sistems it is possible to reach 4k videos running at 60 frames per second, whereas the best neural network solution is able to handle 224x224 frames at 300 milliseconds in the same embedded system. Due to this, a vast number of solutions were developed: dedicated hardware for inference process; models manipulation; accelerated pre-processing image solutions; and videos manupalation techniques. This paper is based at the study of these videos manipulation techniques, exposing the most common algorithmics, such as frame-skip, frame-droop, resizing, color convert, and overlay solutions; and preseting the benneficies and issues of the adoption or not of those techniques.*



## Introduction

Aiming to diminish the gap between machine learning inference process time and the multimedia capability, which reachs 4k@60fps, some video manipulation solutions was purposed. Among them, the most common are the overlay solutions, which are able to create alpha layers over the video and insert information on it. These overlays are very common on object detections, once are responsible for drawing a square, select or color an object at the scene. The most common overlays are SVG, Cairo and OpenCV.
As the overlays, it is important to use a great video framework able to handle all the elements involved at the inference and display solutions. One of the best and most usage one is the GStreamer framework. GStreamer is able to handle plugins in pipelines, which is perfect to do some tests very quickly.
Appart from the video solutions, it is important to choose the best machine learning algorithmics as well. With focus on object detection, the most common and valueted ones are Single Shot Detection (SSD) and Tensorflow. Both as a incridible inference process capability, and the TFlite version demonstrated a great tool for embedded systems.
Thus, with all the tools available, this paper intends to compare the combination of theses algorithmics for object detection solutions. This comparison aims to demonstrate how we can increase the video frame rate with simple approaches and demonstrate the best scenarios to handle each neural network algorithmics and the overlays plugins behavior on these tests.

## Materials and Methods

This section describes the material such as the video files, models and programming language used, and adopted methodology.

### Programming Language



This paper uses Python 3 language and all the required support for the overlays, GStreamer, Tensorflow and SSD. These tools will be described in details on this section.
As first test, it will uses opencv to open a video based file in a indepent new window.

In [6]:
import cv2 as opencv

In [9]:
window="Video test"
opencv.namedWindow(window)
file="../figures/video_device.mp4"
cap=opencv.VideoCapture(file)
    
while (cap.isOpened()):
    ret,frame=cap.read()
        
    if ret:
        opencv.imshow(window,frame)
        if opencv.waitKey(133)==27:
            break
    else :
        break
            
opencv.destroyAllWindows()
cap.release()

## Results

The following results.

## Conclusion

This is the conclusion:

## Source

Hwang, C. & Yoon, K., 1981. Multiple Attributes Decision Making Methods and Applications. Berlin: Springer-Verlag.